In [1]:
import numpy as np
import pandas as pd

import datetime

from elo import *

# import nfl_data_py as nfl  # TODO: incorporate API

In [11]:
# Read historical data

df = pd.read_csv('data/archive/spreadspoke_scores.csv')

df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN


In [4]:
df['schedule_date'] = pd.to_datetime(df['schedule_date']).dt.date
df = df[df['schedule_date'] > datetime.date(1999, 1, 1)]

In [5]:
df['team_home'] = df['team_home'].str.split(' ').str[-1]
df['team_away'] = df['team_away'].str.split(' ').str[-1]
df.head()

/var/folders/cy/yzj3l7550594tg8c768zvjkr0000gp/T/ipykernel_42879/1691658862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['team_home'] = df['team_home'].str.split(' ').str[-1]
/var/folders/cy/yzj3l7550594tg8c768zvjkr0000gp/T/ipykernel_42879/1691658862.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['team_away'] = df['team_away'].str.split(' ').str[-1]


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
7084,1999-01-02,1998,Wildcard,True,Cowboys,7,20,Cardinals,DAL,-7.0,42,Texas Stadium,False,NaN,NaN,NaN,NaN
7085,1999-01-02,1998,Wildcard,True,Dolphins,24,17,Bills,MIA,-2.5,43,Sun Life Stadium,False,NaN,NaN,NaN,NaN
7086,1999-01-03,1998,Wildcard,True,Jaguars,25,10,Patriots,JAX,-8.5,41,EverBank Field,False,NaN,NaN,NaN,NaN
7087,1999-01-03,1998,Wildcard,True,49ers,30,27,Packers,SF,-3.0,49.5,Candlestick Park,False,NaN,NaN,NaN,NaN
7088,1999-01-09,1998,Division,True,Falcons,20,18,49ers,ATL,-3.5,53,Georgia Dome,False,72.0,0.0,NaN,indoor


In [6]:
# Retroactively correct re-named teams
df.replace(to_replace="Team", value="Commanders", inplace=True)  # Washington Football Team -> Washington Commanders
df.replace(to_replace='Redskins', value='Commanders', inplace=True)
df.replace(to_replace='Oilers', value='Titans', inplace=True)

teams = df['team_home'].unique()
print(f"{len(teams)} total teams")  # (should be 32)

32 total teams


/var/folders/cy/yzj3l7550594tg8c768zvjkr0000gp/T/ipykernel_42879/2047392528.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace(to_replace="Team", value="Commanders", inplace=True)  # Washington Football Team -> Washington Commanders
/var/folders/cy/yzj3l7550594tg8c768zvjkr0000gp/T/ipykernel_42879/2047392528.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace(to_replace='Redskins', value='Commanders', inplace=True)
/var/folders/cy/yzj3l7550594tg8c768zvjkr0000gp/T/ipykernel_42879/2047392528.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

In [7]:
# Initialize Elo
elo = {team : 1500 for team in teams}

This system is based of the [elo system used by FiveThirtyEight](https://fivethirtyeight.com/methodology/how-our-nfl-predictions-work/). Elo is a score assigned to each team that represents their team-strength. It is derived from that team's performance in a closed pool of zero-sum games. 

Using elo, you can calculate the odds of team A winning a given game as:

\begin{equation*}
Pr(A) = \frac{1}{10^{\frac{-Elo Diff}{400}} + 1}
\end{equation*}



Here are the main features that go into updating elo scores in this system:

# The K-factor

This is a parameter that controls how quickly elo updates according to recent games. A high K-factor indicates that 

# The forecast delta 

This is how different the outcome of the game is from what the Elo system predicted.

# Margin-of-victory multiplier

\begin{equation*}
Mov Multiplier = \ln{(Winner Point Diff+1)} \times \frac{2.2}{Winner Elo Diff \times 0.001 + 2.2}
\end{equation*}

In [10]:
# for example... update_elo(1509, 1757, 21, 20)